In [13]:
# Check GPU availability
!nvidia-smi
import torch
print("torch cuda:", torch.cuda.is_available(), torch.version.cuda)



/bin/bash: line 1: nvidia-smi: command not found
torch cuda: False 12.6
torch cuda: False 12.6


In [ ]:
# Mount Google Drive (chỉ chạy trên Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IS_COLAB = True
    print("Đang chạy trên Google Colab")
except:
    IS_COLAB = False
    print("Đang chạy local (không phải Colab)")

ℹ️ Đang chạy local (không phải Colab)


In [15]:
# Set paths (tự động điều chỉnh cho local hoặc Colab)
import os
import sys

if IS_COLAB:
    ROOT = "/content/FaceRecognition"
    DRIVE_ROOT = "/content/drive/MyDrive/FaceRecognition"
else:
    # Local - check if running on Windows path (D:\ drive)
    # Hardcode vì kernel có thể chạy trên WSL
    ROOT = r"D:\HCMUTE_project\DIP\FaceRecognition"
    # Nếu D:\ không tồn tại, fallback to current directory
    if not os.path.exists(ROOT):
        ROOT = os.path.dirname(os.path.abspath(os.getcwd()))
        if not os.path.exists(os.path.join(ROOT, 'configs')):
            ROOT = os.getcwd()
    DRIVE_ROOT = None
    
if DRIVE_ROOT:
    os.makedirs(DRIVE_ROOT, exist_ok=True)
    
print("Platform:", sys.platform)
print("ROOT=", ROOT)
print("DRIVE_ROOT=", DRIVE_ROOT)
print(f"ROOT exists: {os.path.exists(ROOT)}")
print(f"Configs exists: {os.path.exists(os.path.join(ROOT, 'configs'))}")

Platform: linux
ROOT= /content
DRIVE_ROOT= None
ROOT exists: True
Configs exists: False


In [16]:
# Clone/Update repository (chỉ trên Colab)
if IS_COLAB:
    REPO_URL = "https://github.com/sin0235/FaceRecognition.git"  
    if not os.path.exists(os.path.join(ROOT, ".git")):
        !git clone $REPO_URL $ROOT
    else:
        %cd $ROOT
        !git pull
    %cd $ROOT
else:
    print("Đang chạy local, bỏ qua clone repo")
    print(f"Working directory: {ROOT}")

Đang chạy local, bỏ qua clone repo
Working directory: /content


In [ ]:
# Sync dữ liệu từ Drive (chỉ trên Colab)
if IS_COLAB and DRIVE_ROOT:
    import shutil
    if os.path.exists(os.path.join(DRIVE_ROOT, "data")):
        print("Đang sync dữ liệu từ Drive...")
        shutil.copytree(os.path.join(DRIVE_ROOT, "data"), 
                        os.path.join(ROOT, "data"), 
                        dirs_exist_ok=True)
        print("Sync hoàn tất")
    else:
        print("Chưa có dữ liệu trong Drive, bỏ qua sync")
else:
    print("Đang chạy local, bỏ qua sync từ Drive")

ℹ️ Đang chạy local, bỏ qua sync từ Drive


In [ ]:
# Install dependencies (chỉ trên Colab)
if IS_COLAB:
    %pip install -q torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
    %pip install -q mxnet-cu118==1.9.1 onnxruntime-gpu==1.16.0 insightface==0.7.3
    %pip install -q -r requirements.txt
else:
    print("Đang chạy local, bỏ qua cài đặt dependencies")
    print("Đảm bảo bạn đã cài đặt: pip install -r requirements.txt")

ℹ️ Đang chạy local, bỏ qua cài đặt dependencies
Đảm bảo bạn đã cài đặt: pip install -r requirements.txt


In [ ]:
# Quick GPU sanity checks
import torch
print("torch cuda:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("  - CUDA version:", torch.version.cuda)
    print("  - Device name:", torch.cuda.get_device_name(0))
    print("  - Device count:", torch.cuda.device_count())
else:
    print("  - CUDA không khả dụng, sẽ sử dụng CPU")

# Kiểm tra mxnet và onnxruntime (optional, chỉ cần nếu dùng InsightFace)
try:
    import mxnet as mx
    print("mxnet gpus:", mx.context.num_gpus())
except ImportError:
    print("mxnet: chưa cài đặt (optional)")

try:
    import onnxruntime as ort
    print("onnxruntime device:", ort.get_device())
except ImportError:
    print("onnxruntime: chưa cài đặt (optional)")

torch cuda: False
  - ⚠️ CUDA không khả dụng, sẽ sử dụng CPU
mxnet: chưa cài đặt (optional)
onnxruntime: chưa cài đặt (optional)


In [20]:
# Prepare data/checkpoints paths
if IS_COLAB and DRIVE_ROOT:
    CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, "models", "checkpoints")
    DATA_DIR = os.path.join(DRIVE_ROOT, "data")
else:
    # Local paths
    CHECKPOINT_DIR = os.path.join(ROOT, "models", "checkpoints")
    DATA_DIR = os.path.join(ROOT, "data")
    
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
print("CHECKPOINT_DIR=", CHECKPOINT_DIR)
print("DATA_DIR=", DATA_DIR)

CHECKPOINT_DIR= /content/models/checkpoints
DATA_DIR= /content/data


In [21]:
# Cấu hình đường dẫn và kiểm tra files
import os
import sys

# List ROOT directory
print("=== Nội dung ROOT ===")
print(f"ROOT: {ROOT}")
if os.path.exists(ROOT):
    items = sorted(os.listdir(ROOT))
    print(f"Found {len(items)} items:")
    for item in items[:25]:
        print(f"  - {item}")
else:
    print("ROOT không tồn tại!")

# Config paths
CONFIG_PATH = os.path.join(ROOT, "configs", "arcface_config.yaml")
PRETRAINED_BACKBONE = os.path.join(CHECKPOINT_DIR, "ms1mv2_resnet50.pth")

print(f"\n=== File Paths ===")
print(f"CONFIG_PATH: {CONFIG_PATH}")
print(f"Config exists: {os.path.exists(CONFIG_PATH)}")
print(f"PRETRAINED: {PRETRAINED_BACKBONE}")
print(f"Pretrained exists: {os.path.exists(PRETRAINED_BACKBONE)}")

=== Nội dung ROOT ===
ROOT: /content
Found 6 items:
  - .config
  - FaceRecognition
  - data
  - drive
  - models
  - sample_data

=== File Paths ===
CONFIG_PATH: /content/configs/arcface_config.yaml
Config exists: False
PRETRAINED: /content/models/checkpoints/ms1mv2_resnet50.pth
Pretrained exists: False


In [ ]:
# Kiểm tra dữ liệu
import pandas as pd

data_processed_dir = os.path.join(ROOT, "data", "processed")
train_csv_path = os.path.join(data_processed_dir, "train_metadata.csv")
val_csv_path = os.path.join(data_processed_dir, "val_metadata.csv")

print("=== KIỂM TRA DỮ LIỆU ===\n")

# Kiểm tra train metadata
if os.path.exists(train_csv_path):
    train_df = pd.read_csv(train_csv_path)
    print(f"Train metadata: {len(train_df)} ảnh")
    print(f"  - Số identities: {train_df['identity_name'].nunique()}")
    print(f"  - Columns: {list(train_df.columns)}")
    print(f"\nSample:")
    print(train_df.head(3))
else:
    print(f"Không tìm thấy: {train_csv_path}")

print("\n" + "-"*50 + "\n")

# Kiểm tra val metadata
if os.path.exists(val_csv_path):
    val_df = pd.read_csv(val_csv_path)
    print(f"Val metadata: {len(val_df)} ảnh")
    print(f"  - Số identities: {val_df['identity_name'].nunique()}")
else:
    print(f"Không tìm thấy: {val_csv_path}")

print("\n" + "-"*50 + "\n")

# Kiểm tra thư mục ảnh
train_dir = os.path.join(data_processed_dir, "train")
val_dir = os.path.join(data_processed_dir, "val")

if os.path.exists(train_dir):
    train_folders = [f for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))]
    print(f"Train images dir: {len(train_folders)} folders")
else:
    print(f"Không tìm thấy: {train_dir}")

if os.path.exists(val_dir):
    val_folders = [f for f in os.listdir(val_dir) if os.path.isdir(os.path.join(val_dir, f))]
    print(f"Val images dir: {len(val_folders)} folders")
else:
    print(f"Không tìm thấy: {val_dir}")

=== KIỂM TRA DỮ LIỆU ===

❌ Không tìm thấy: /content/data/processed/train_metadata.csv

--------------------------------------------------

❌ Không tìm thấy: /content/data/processed/val_metadata.csv

--------------------------------------------------

❌ Không tìm thấy: /content/data/processed/train
❌ Không tìm thấy: /content/data/processed/val


## Kiểm Tra Dữ Liệu

Trước khi training, hãy kiểm tra xem dữ liệu đã sẵn sàng chưa.

In [ ]:
# Kiểm tra và chạy training
TRAIN_SCRIPT = os.path.join(ROOT, "models", "arcface", "train_arcface.py")

if not os.path.exists(TRAIN_SCRIPT):
    print(f"Thiếu training script: {TRAIN_SCRIPT}")
else:
    print(f"Training script tồn tại: {TRAIN_SCRIPT}")
    
    # Kiểm tra dữ liệu
    train_csv = os.path.join(ROOT, "data", "processed", "train_metadata.csv")
    val_csv = os.path.join(ROOT, "data", "processed", "val_metadata.csv")
    
    if not os.path.exists(train_csv):
        print(f"Chưa có dữ liệu training: {train_csv}")
        print("Để tiếp tục, cần chuẩn bị:")
        print("  1. Upload dữ liệu đã preprocess lên Drive")
        print(f"  2. Đảm bảo có file: {os.path.join(DATA_DIR, 'processed/train_metadata.csv')}")
        print("  3. Chạy lại cell 5 để sync dữ liệu")
    elif not os.path.exists(val_csv):
        print(f"Chưa có dữ liệu validation: {val_csv}")
        print("Vui lòng chuẩn bị dữ liệu trước khi training!")
    else:
        print("\n" + "="*50)
        print("BẮT ĐẦU TRAINING")
        print("="*50)
        
        # Chạy training với đầy đủ arguments
        !python $TRAIN_SCRIPT \
            --config $CONFIG_PATH \
            --pretrained_backbone $PRETRAINED_BACKBONE \
            --data_dir $ROOT/data \
            --checkpoint_dir $CHECKPOINT_DIR

❌ Thiếu training script: /content/models/arcface/train_arcface.py


In [24]:
# Test model sau khi training
import sys
sys.path.append(ROOT)

from models.arcface.arcface_model import test_model

print("Testing ArcFace Model...")
test_model()

ModuleNotFoundError: No module named 'models.arcface'

In [ ]:
# Load best checkpoint và extract embeddings
import torch
from models.arcface.arcface_model import ArcFaceModel

checkpoint_path = os.path.join(CHECKPOINT_DIR, "arcface_best.pth")

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint: {checkpoint_path}")
    
    checkpoint = torch.load(checkpoint_path)
    num_classes = len(checkpoint['config']['data'].get('num_classes', 100))
    
    model = ArcFaceModel(num_classes=num_classes, embedding_size=512)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"Loaded best model - Epoch {checkpoint['epoch']}")
    print(f"Validation accuracy: {checkpoint['val_acc']:.2f}%")
    
    # Test inference
    dummy_input = torch.randn(1, 3, 112, 112)
    with torch.no_grad():
        embedding = model.extract_features(dummy_input)
    
    print(f"Embedding shape: {embedding.shape}")
    print("Model sẵn sàng cho inference!")
else:
    print(f"Không tìm thấy checkpoint: {checkpoint_path}")